In [1]:
# import os
# os.environ['KAGGLE_USERNAME'] = 'aashish31476'
# os.environ['KAGGLE_KEY'] = '306e7cb470122dd3f86d4c7db6d197a9'
# !kaggle competitions files leash-BELKA

name                    size  creationDate         
---------------------  -----  -------------------  
sample_submission.csv   22MB  2024-03-20 23:16:46  
test.csv               297MB  2024-03-20 23:16:46  
train.parquet            3GB  2024-03-20 23:16:46  
train.csv               50GB  2024-03-20 23:16:46  
test.parquet            29MB  2024-03-20 23:16:46  


In [2]:
# !kaggle competitions download leash-BELKA -f test.parquet
# !unzip test.parquet.zip

 50% 9.00M/18.2M [00:00<00:00, 32.6MB/s]
100% 18.2M/18.2M [00:00<00:00, 54.0MB/s]
Archive:  test.parquet.zip
  inflating: test.parquet            


In [3]:
# !kaggle competitions download leash-BELKA -f train.parquet
# !unzip train.parquet.zip

In [1]:
# !pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.7/32.7 MB 46.2 MB/s eta 0:00:0000:0100:01


In [36]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski, Atom

from padelpy import from_smiles

In [33]:
full_df = pd.read_parquet('test.parquet')
full_df

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name
0,295246830,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,BRD4
1,295246831,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,HSA
2,295246832,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,sEH
3,295246833,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,CC(O)Cn1cnc2c(N)ncnc21,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ncnc3c2...,BRD4
4,295246834,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,CC(O)Cn1cnc2c(N)ncnc21,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ncnc3c2...,HSA
...,...,...,...,...,...,...
1674891,296921721,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,Nc1noc2ccc(F)cc12,COC1CCC(CCN)CC1,COC1CCC(CCNc2nc(Nc3noc4ccc(F)cc34)nc(N[C@@H](C...,HSA
1674892,296921722,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,Nc1noc2ccc(F)cc12,COC1CCC(CCN)CC1,COC1CCC(CCNc2nc(Nc3noc4ccc(F)cc34)nc(N[C@@H](C...,sEH
1674893,296921723,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,Nc1noc2ccc(F)cc12,NCc1cccs1,[N-]=[N+]=NCCC[C@H](Nc1nc(NCc2cccs2)nc(Nc2noc3...,BRD4
1674894,296921724,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,Nc1noc2ccc(F)cc12,NCc1cccs1,[N-]=[N+]=NCCC[C@H](Nc1nc(NCc2cccs2)nc(Nc2noc3...,HSA


In [ ]:
df = full_df.sample(100, random_state=42)
df

# Calculate Lipinski descriptors

Christopher Lipinski, a scientist at Pfizer, came up with a set of rule-of-thumb for evaluating the druglikeness of compounds. Such druglikeness is based on the Absorption, Distribution, Metabolism and Excretion (ADME) that is also known as the pharmacokinetic profile. Lipinski analyzed all orally active FDA-approved drugs in the formulation of what is to be known as the Rule-of-Five or Lipinski's Rule.

The Lipinski's Rule stated the following:

1. Molecular weight < 500 Dalton
2. Octanol-water partition coefficient (LogP) < 5
3. Hydrogen bond donors < 5
4. Hydrogen bond acceptors < 10

In [53]:
def lipinski(smiles, df=True, verbose=False):

    moldata= []
    for elem in tqdm(smiles):
        mol=Chem.MolFromSmiles(elem)
        moldata.append(mol)

    baseData= np.arange(1,1)
    i=0
    for mol in moldata:

        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)

        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])

        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1

    if df:
        columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]
        descriptors = pd.DataFrame(data=baseData,columns=columnNames)
        return descriptors
    else:
        return baseData

In [54]:
lipinski(df.buildingblock1_smiles, df=True, verbose=True)

  0%|          | 0/100 [00:00<?, ?it/s]

,MW,LogP,NumHDonors,NumHAcceptors
0,379.456,4.4161,2.0,3.0
1,355.822,2.8788,1.0,5.0
2,440.253,4.2983,2.0,5.0
3,442.496,5.8693,2.0,5.0
4,384.270,3.8024,1.0,3.0
...,...,...,...,...
95,435.907,5.2166,1.0,3.0
96,401.462,4.6111,2.0,3.0
97,407.491,4.6726,2.0,4.0
98,429.259,5.4476,2.0,4.0


# Calculate fingerprint descriptors

## Calculate PaDEL descriptors

In [7]:
# !git clone https://github.com/dataprofessor/bioinformatics.git
# !unzip ./bioinformatics/padel.zip

In [17]:
df.head(1)

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name
625149,295871979,O=C(NCC1CCC(C(=O)O)CC1)OCC1c2ccccc2-c2ccccc21,COc1cccc(-c2cc(N)on2)c1,CC(C)(C)OC(=O)N1Cc2c(N)n[nH]c2C1(C)C,COc1cccc(-c2cc(Nc3nc(NCC4CCC(C(=O)N[Dy])CC4)nc...,sEH


In [18]:
selection = ['buildingblock1_smiles']
df_selection = df[selection]
df_selection.to_csv('buildingblock1_smiles.smi', sep='\t', index=False, header=False)

In [19]:
!cat buildingblock1_smiles.smi | head -5

O=C(NCC1CCC(C(=O)O)CC1)OCC1c2ccccc2-c2ccccc21
CN(c1cc(C(=O)O)cc(Cl)n1)C1CCN(C(=O)OC(C)(C)C)C1
O=C(Nc1ncc(Br)nc1C(=O)O)OCC1c2ccccc2-c2ccccc21
O=C(Nc1nc(-c2ccc(C(=O)O)cc2)cs1)OCC1c2ccccc2-c2ccccc21
CC(C)(C)OC(=O)N1CCC(C(=O)O)(c2cccc(Br)c2)CC1


In [23]:
!cat /content/bioinformatics/padel.sh

java -Xms1G -Xmx1G -Djava.awt.headless=true -jar ./PaDEL-Descriptor/PaDEL-Descriptor.jar -removesalt -standardizenitro -fingerprints -descriptortypes ./PaDEL-Descriptor/PubchemFingerprinter.xml -dir ./ -file descriptors_output.csv


In [34]:
!du -sh /content/descriptors_output.csv /content/descriptors_output.parquet

188K	/content/descriptors_output.csv
188K	/content/descriptors_output.parquet


In [9]:
!java -Xms1G -Xmx1G -Djava.awt.headless=true -jar ./PaDEL-Descriptor/PaDEL-Descriptor.jar -removesalt -standardizenitro -fingerprints -descriptortypes ./PaDEL-Descriptor/PubchemFingerprinter.xml -dir ./ -file descriptors_output.csv -maxcpdperfile 20 -threads -1 -usefilenameasmolname smiles.smi -waitingjobs -1

Processing smiles_1 in smiles.smi (1/1000). 
Processing smiles_2 in smiles.smi (2/1000). 
Processing smiles_3 in smiles.smi (3/1000). 
Processing smiles_4 in smiles.smi (4/1000). 
Processing smiles_5 in smiles.smi (5/1000). 
Processing smiles_6 in smiles.smi (6/1000). 
Processing smiles_7 in smiles.smi (7/1000). 
Processing smiles_8 in smiles.smi (8/1000). 
Processing smiles_9 in smiles.smi (9/1000). 
Processing smiles_10 in smiles.smi (10/1000). 
Processing smiles_11 in smiles.smi (11/1000). 
Processing smiles_12 in smiles.smi (12/1000). 
Processing smiles_13 in smiles.smi (13/1000). 
Processing smiles_14 in smiles.smi (14/1000). 
Processing smiles_15 in smiles.smi (15/1000). 
Processing smiles_16 in smiles.smi (16/1000). 
Processing smiles_17 in smiles.smi (17/1000). 
Processing smiles_18 in smiles.smi (18/1000). 
Processing smiles_19 in smiles.smi (19/1000). 
Processing smiles_20 in smiles.smi (20/1000). 
Processing smiles_21 in smiles.smi (21/1000). 
Processing smiles_22 in smiles.

In [ ]:
! bash /content/bioinformatics/padel.sh

In [ ]:
!head /content/descriptors_output.csv

In [26]:
import subprocess

bash_file = '/content/bioinformatics/padel.sh'

result = subprocess.run(['bash', bash_file], capture_output=False, text=True)

# Print output and errors (if any)
print("Output:", result.stdout)
print("Error:", result.stderr)

Output: None
Error: None


In [41]:
def padel(smiles, output_name):
    smiles.to_csv('smiles.smi', sep='\t', index=False, header=False)
    command = [
    'java', '-Xms1G', '-Xmx1G', '-Djava.awt.headless=true',
    '-jar', './PaDEL-Descriptor/PaDEL-Descriptor.jar',
    '-removesalt', '-standardizenitro', '-fingerprints',
    '-descriptortypes', './PaDEL-Descriptor/PubchemFingerprinter.xml',
    '-dir', './', '-file', f'{output_name}.csv'
    ]
    subprocess.run(command, capture_output=False, text=True)

In [37]:
df_selection

,buildingblock1_smiles
625149,O=C(NCC1CCC(C(=O)O)CC1)OCC1c2ccccc2-c2ccccc21
372795,CN(c1cc(C(=O)O)cc(Cl)n1)C1CCN(C(=O)OC(C)(C)C)C1
1268175,O=C(Nc1ncc(Br)nc1C(=O)O)OCC1c2ccccc2-c2ccccc21
1237758,O=C(Nc1nc(-c2ccc(C(=O)O)cc2)cs1)OCC1c2ccccc2-c...
95055,CC(C)(C)OC(=O)N1CCC(C(=O)O)(c2cccc(Br)c2)CC1
...,...
363192,CN(C(=O)OCC1c2ccccc2-c2ccccc21)[C@@H](Cc1ccc(C...
572596,O=C(NC(CCc1ccccc1)C(=O)O)OCC1c2ccccc2-c2ccccc21
1362786,O=C(O)C[C@@H](Cc1cccs1)NC(=O)OCC1c2ccccc2-c2cc...
936875,O=C(Nc1c(Cl)cc(Cl)nc1C(=O)O)OCC1c2ccccc2-c2ccc...


In [43]:
padel(df_selection, 'buildingblock1_smiles')

In [44]:
pd.read_csv('/content/descriptors_output.csv')

,Name,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,AUTOGEN_smiles_2,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AUTOGEN_smiles_1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AUTOGEN_smiles_3,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AUTOGEN_smiles_5,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AUTOGEN_smiles_4,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,AUTOGEN_smiles_96,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,AUTOGEN_smiles_97,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,AUTOGEN_smiles_98,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,AUTOGEN_smiles_99,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Compailed Function

In [19]:
def lipinski(smiles, df=True):
    if type(smiles).__name__ == 'DataFrame':
        smiles = smiles.values.flatten()

    moldata= []
    for elem in tqdm(smiles):
        mol=Chem.MolFromSmiles(elem)
        moldata.append(mol)

    baseData= np.arange(1,1)
    i=0
    for mol in moldata:

        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)

        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])

        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1

    if df:
        columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]
        descriptors = pd.DataFrame(data=baseData,columns=columnNames)
        return descriptors
    else:
        return baseData

def lipinski3(smiles, df=True):
    # if type(smiles).__name__ == 'DataFrame':
    #     smiles = smiles.values.flatten()

    moldata= []
    baseData= np.arange(1,1)
    i=0
    for elem in tqdm(smiles):
        mol=Chem.MolFromSmiles(elem)

        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)

        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])

        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1

    if df:
        columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]
        descriptors = pd.DataFrame(data=baseData,columns=columnNames)
        return descriptors
    else:
        return baseData

from joblib import Parallel, delayed
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import subprocess

def compute_descriptors(smiles):
    """Compute molecular descriptors for a single SMILES string."""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.array([np.nan, np.nan, np.nan, np.nan])  # Handle invalid SMILES

    desc_MolWt = Descriptors.MolWt(mol)
    desc_MolLogP = Descriptors.MolLogP(mol)
    desc_NumHDonors = Lipinski.NumHDonors(mol)
    desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)

    return np.array([desc_MolWt, desc_MolLogP, desc_NumHDonors, desc_NumHAcceptors])

def lipinski_parallel(smiles, df=True, n_jobs=-1):
    """Compute Lipinski descriptors in parallel."""
    if isinstance(smiles, pd.DataFrame):
        smiles = smiles.values.flatten()

    # Compute descriptors in parallel
    results = Parallel(n_jobs=n_jobs, backend='loky')(
        delayed(compute_descriptors)(s) for s in tqdm(smiles)
    )

    # Stack results into a NumPy array
    baseData = np.vstack(results)

    if df:
        columnNames = ["MW", "LogP", "NumHDonors", "NumHAcceptors"]
        descriptors = pd.DataFrame(data=baseData, columns=columnNames)
        return descriptors
    else:
        return baseData

# Example usage
# smiles_list = ['CCO', 'CCN', 'CCC']
# result_df = lipinski_parallel(smiles_list)
# print(result_df)


def padel(smiles, output_name):
    smiles.to_csv('smiles.smi', sep='\t', index=False, header=False)
    command = [
    'java', '-Xms5G', '-Xmx5G', '-Djava.awt.headless=true',
    '-jar', './PaDEL-Descriptor/PaDEL-Descriptor.jar',
    '-removesalt', '-standardizenitro', '-fingerprints',
    '-descriptortypes', './PaDEL-Descriptor/PubchemFingerprinter.xml',
    '-dir', './', '-file', f'{output_name}.csv'
    ]
    subprocess.run(command, capture_output=False, text=True)


def get_descriptors(smiles, output_name, df=True):
    # lipinski_df = lipinski(smiles, df=True)
    lipinski_df = lipinski_parallel(smiles, df=True)
    padel(smiles, f"{output_name}_descriptors")
    padel_df = pd.read_csv(f"{output_name}_descriptors.csv")
    df = pd.concat([lipinski_df, padel_df], axis=1)
    return df

In [5]:
full_df = pd.read_parquet('test.parquet')

In [8]:
df = full_df.sample(10000, random_state=42)
df

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name
625149,295871979,O=C(NCC1CCC(C(=O)O)CC1)OCC1c2ccccc2-c2ccccc21,COc1cccc(-c2cc(N)on2)c1,CC(C)(C)OC(=O)N1Cc2c(N)n[nH]c2C1(C)C,COc1cccc(-c2cc(Nc3nc(NCC4CCC(C(=O)N[Dy])CC4)nc...,sEH
372795,295619625,CN(c1cc(C(=O)O)cc(Cl)n1)C1CCN(C(=O)OC(C)(C)C)C1,Cc1cocc1B1OC(C)(C)C(C)(C)O1,CC(C)(C)C(=O)Nc1cc(C(=O)O)ccn1,Cc1cocc1-c1cc(C(=O)N[Dy])cc(N(C)C2CCN(C(=O)c3c...,HSA
1268175,296515005,O=C(Nc1ncc(Br)nc1C(=O)O)OCC1c2ccccc2-c2ccccc21,Cl.NCC1CCOCC12CCCC2,Cc1ccc(Cl)c(N)c1,Cc1ccc(Cl)c(Nc2nc(NCC3CCOCC34CCCC4)nc(Nc3ncc(B...,HSA
1237758,296484588,O=C(Nc1nc(-c2ccc(C(=O)O)cc2)cs1)OCC1c2ccccc2-c...,Cl.NCCc1cc(Br)c(Br)s1,Cl.Cn1cc(CN)c(=O)[nH]c1=O,Cn1cc(CNc2nc(NCCc3cc(Br)c(Br)s3)nc(Nc3nc(-c4cc...,HSA
95055,295341885,CC(C)(C)OC(=O)N1CCC(C(=O)O)(c2cccc(Br)c2)CC1,Cc1ccc(NS(C)(=O)=O)cc1B1OC(C)(C)C(C)(C)O1,CCCc1cc(C(=O)O)nn1C,CCCc1cc(C(=O)N2CCC(C(=O)N[Dy])(c3cccc(-c4cc(NS...,BRD4
...,...,...,...,...,...,...
1080400,296327230,O=C(Nc1ccc(Br)cc1C(=O)O)OCC1c2ccccc2-c2ccccc21,Nc1nc(Cl)cc(Cl)n1,Cl.NCC1(O)C2C3CC4C5C3CC2C5C41,O=C(N[Dy])c1cc(Br)ccc1Nc1nc(NCC2(O)C3C4CC5C6C4...,sEH
1039681,296286511,O=C(Nc1cc(Cl)ncc1C(=O)O)OCC1c2ccccc2-c2ccccc21,CNC(=O)C(O)CN.Cl,Cl.NCCCNC(=O)c1ccc(F)cc1,CNC(=O)C(O)CNc1nc(NCCCNC(=O)c2ccc(F)cc2)nc(Nc2...,sEH
1597959,296844789,O=C(O)c1cc(Br)cnc1N1CCCN(C(=O)OCC2c3ccccc3-c3c...,O=C(NC1CC1)c1ccc(B(O)O)cc1,O=C(O)C1CCC2(CC1)CC2,O=C(NC1CC1)c1ccc(-c2cnc(N3CCCN(C(=O)C4CCC5(CC4...,HSA
105289,295352119,CC(C)(C)OC(=O)N1CCC(COc2cc(I)ccc2C(=O)O)CC1,CCCOc1ccc(C(F)(F)F)cc1B(O)O,O=C(O)[C@@H]1Cc2ccccc2O[C@H]1C(F)F,CCCOc1ccc(C(F)(F)F)cc1-c1ccc(C(=O)N[Dy])c(OCC2...,sEH


In [17]:
def lipinski2(smiles, df=True):
    if type(smiles).__name__ == 'DataFrame':
        smiles = smiles.values.flatten()

    moldata = []
    for elem in tqdm(smiles):
        mol = Chem.MolFromSmiles(elem)
        moldata.append(mol)

    baseData = np.arange(1, 1)
    i = 0
    for mol in moldata:
        if mol is None:
            continue  # Skip invalid molecules

        # General Chemical Descriptors
        desc_MolWt = Descriptors.MolWt(mol)  # Molecular Weight (MW)
        desc_ExactMolWt = rdMolDescriptors.CalcExactMolWt(mol)  # Exact Molecular Weight
        desc_MolLogP = Descriptors.MolLogP(mol)  # LogP (Hydrophobicity)
        desc_TPSA = rdMolDescriptors.CalcTPSA(mol)  # Topological Polar Surface Area (TPSA)
        desc_NumHDonors = Lipinski.NumHDonors(mol)  # Number of H-Bond Donors
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)  # Number of H-Bond Acceptors
        desc_NumRotatableBonds = rdMolDescriptors.CalcNumRotatableBonds(mol)  # Number of Rotatable Bonds
        desc_FractionCSP3 = rdMolDescriptors.CalcFractionCSP3(mol)  # Fraction of Csp3 Carbons
        desc_NumHeavyAtoms = mol.GetNumHeavyAtoms()  # Number of Heavy Atoms
        
        # Functional Group Descriptors
        desc_NumAromaticRings = rdMolDescriptors.CalcNumAromaticRings(mol)  # Number of Aromatic Rings
        desc_NumAliphaticRings = rdMolDescriptors.CalcNumAliphaticRings(mol)  # Number of Aliphatic Rings
        desc_NumSaturatedRings = rdMolDescriptors.CalcNumSaturatedRings(mol)  # Number of Saturated Rings
        desc_NumHeteroatoms = rdMolDescriptors.CalcNumHeteroatoms(mol)  # Number of Heteroatoms
        desc_NumRings = rdMolDescriptors.CalcNumRings(mol)  # Total Number of Rings
        
        # Shape and Geometry Descriptors
        desc_MolVolume = rdMolDescriptors.CalcMolVolume(mol)  # Molecular Volume
        desc_RadiusOfGyration = rdMolDescriptors.CalcRadiusOfGyration(mol)  # Radius of Gyration
        desc_InertialShapeFactor = rdMolDescriptors.CalcInertialShape(mol)  # Inertial Shape Factor
        
        # Binding-Specific Descriptors
        desc_FormalCharge = Descriptors.FormalCharge(mol)  # Formal Charge
        desc_AromaticProportion = (
            rdMolDescriptors.CalcNumAromaticRings(mol) / mol.GetNumHeavyAtoms()
            if mol.GetNumHeavyAtoms() > 0 else 0
        )  # Aromatic Proportion
        desc_HBondPotential = Lipinski.NumHDonors(mol) + Lipinski.NumHAcceptors(mol)  # Hydrogen Bonding Potential
        desc_Lipophilicity = Descriptors.MolLogP(mol)  # Lipophilicity
        desc_ChargeDistribution = Descriptors.FormalCharge(mol)  # Charge Distribution
        
        # Molecular Fingerprints
        fp_RDK = AllChem.RDKFingerprint(mol)  # RDK Fingerprint
        desc_FingerprintBits = sum(fp_RDK.GetOnBits())  # Number of Active Bits in RDK Fingerprint
        fp_Morgan = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024)  # Morgan Fingerprint
        desc_MorganBits = sum(fp_Morgan)  # Number of Active Bits in Morgan Fingerprint
        
        # Electrostatic Properties
        desc_ElectroNegativity = Descriptors.MolMR(mol)  # Molecular Refractivity (Electronegativity)
        desc_AtomCharges = sum(rdMolDescriptors.ComputeGasteigerCharges(mol).values())  # Sum of Atomic Charges


        row = np.array([
            desc_MolWt, desc_ExactMolWt, desc_MolLogP, desc_TPSA, desc_NumHDonors,
            desc_NumHAcceptors, desc_NumRotatableBonds, desc_FractionCSP3,
            desc_NumHeavyAtoms, desc_NumAromaticRings, desc_NumAliphaticRings,
            desc_NumSaturatedRings, desc_NumHeteroatoms, desc_NumRings,
            desc_MolVolume, desc_RadiusOfGyration, desc_InertialShapeFactor,
            desc_FormalCharge, desc_AromaticProportion, desc_HBondPotential,
            desc_Lipophilicity, desc_ChargeDistribution, desc_FingerprintBits,
            desc_MorganBits, desc_ElectroNegativity, desc_AtomCharges
        ])

        if i == 0:
            baseData = row
        else:
            baseData = np.vstack([baseData, row])
        i += 1

    if df:
        columnNames = [
            "MolWt", "ExactMolWt", "MolLogP", "TPSA", "NumHDonors", "NumHAcceptors",
            "NumRotatableBonds", "FractionCSP3", "NumHeavyAtoms", "NumAromaticRings",
            "NumAliphaticRings", "NumSaturatedRings", "NumHeteroatoms", "NumRings",
            "MolVolume", "RadiusOfGyration", "InertialShapeFactor", "FormalCharge",
            "AromaticProportion", "HBondPotential", "Lipophilicity",
            "ChargeDistribution", "FingerprintBits", "MorganBits",
            "ElectroNegativity", "AtomCharges"
        ]
        descriptors = pd.DataFrame(data=baseData, columns=columnNames)
        return descriptors
    else:
        return baseData

def compute_descriptors(smiles):
    """Compute molecular descriptors for a single SMILES string."""
    mol = Chem.MolFromSmiles(smiles)
    # if mol is None:
    #     return np.array([np.nan, np.nan, np.nan, np.nan])  # Handle invalid SMILES

    desc_MolWt = Descriptors.MolWt(mol)
    desc_MolLogP = Descriptors.MolLogP(mol)
    desc_NumHDonors = Lipinski.NumHDonors(mol)
    desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)

    return np.array([desc_MolWt, desc_MolLogP, desc_NumHDonors, desc_NumHAcceptors])

def lipinski_parallel(smiles, df=True, n_jobs=-1):
    """Compute Lipinski descriptors in parallel."""
    # if isinstance(smiles, pd.DataFrame):
    #     smiles = smiles.values.flatten()

    # Compute descriptors in parallel
    results = Parallel(n_jobs=n_jobs, backend='loky')(
        delayed(compute_descriptors)(s) for s in tqdm(smiles)
    )

    # Stack results into a NumPy array
    baseData = np.vstack(results)

    if df:
        columnNames = ["MW", "LogP", "NumHDonors", "NumHAcceptors"]
        descriptors = pd.DataFrame(data=baseData, columns=columnNames)
        return descriptors
    else:
        return baseData
smiles = df.buildingblock1_smiles

In [20]:
%%timeit
lipinski_df = lipinski3(smiles, df=True)

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

6.92 s ± 1.12 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit
lipinski_df = lipinski2(smiles, df=True)

  0%|          | 0/10000 [00:00<?, ?it/s]

NameError: name 'rdMolDescriptors' is not defined

In [16]:
%%timeit
lipinski_df = lipinski(smiles, df=True)

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

3.98 s ± 624 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit
lipinski_df = lipinski_parallel(smiles, df=True)

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [4]:
%%time
get_descriptors(df.buildingblock1_smiles, 'buildingblock1_smiles')

  0%|          | 0/1000 [00:00<?, ?it/s]

Processing AUTOGEN_smiles_1 in smiles.smi (1/1000). 
Processing AUTOGEN_smiles_3 in smiles.smi (3/1000). 
Processing AUTOGEN_smiles_4 in smiles.smi (4/1000). 
Processing AUTOGEN_smiles_2 in smiles.smi (2/1000). 
Processing AUTOGEN_smiles_5 in smiles.smi (5/1000). 
Processing AUTOGEN_smiles_6 in smiles.smi (6/1000). 
Processing AUTOGEN_smiles_7 in smiles.smi (7/1000). 
Processing AUTOGEN_smiles_8 in smiles.smi (8/1000). 
Processing AUTOGEN_smiles_9 in smiles.smi (9/1000). 
Processing AUTOGEN_smiles_10 in smiles.smi (10/1000). 
Processing AUTOGEN_smiles_11 in smiles.smi (11/1000). 
Processing AUTOGEN_smiles_12 in smiles.smi (12/1000). 
Processing AUTOGEN_smiles_13 in smiles.smi (13/1000). 
Processing AUTOGEN_smiles_14 in smiles.smi (14/1000). 
Processing AUTOGEN_smiles_15 in smiles.smi (15/1000). 
Processing AUTOGEN_smiles_16 in smiles.smi (16/1000). 
Processing AUTOGEN_smiles_17 in smiles.smi (17/1000). 
Processing AUTOGEN_smiles_18 in smiles.smi (18/1000). 
Processing AUTOGEN_smiles_19

,MW,LogP,NumHDonors,NumHAcceptors,Name,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,379.456,4.4161,2.0,3.0,AUTOGEN_smiles_35,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,355.822,2.8788,1.0,5.0,AUTOGEN_smiles_39,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,440.253,4.2983,2.0,5.0,AUTOGEN_smiles_28,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,442.496,5.8693,2.0,5.0,AUTOGEN_smiles_2,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,384.270,3.8024,1.0,3.0,AUTOGEN_smiles_20,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,400.273,3.0358,2.0,5.0,AUTOGEN_smiles_965,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
996,449.894,4.1605,2.0,5.0,AUTOGEN_smiles_989,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
997,417.417,4.5324,2.0,5.0,AUTOGEN_smiles_1000,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
998,431.270,3.8589,1.0,3.0,AUTOGEN_smiles_977,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
file_path = './PaDEL-Descriptor/PubchemFingerprinter.xml'

with open(file_path, 'r') as file:
    content = file.read()

print(content)


In [58]:
get_descriptors(df.buildingblock1_smiles, 'buildingblock1_smiles')

  0%|          | 0/100 [00:00<?, ?it/s]

,MW,LogP,NumHDonors,NumHAcceptors,Name,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,379.456,4.4161,2.0,3.0,AUTOGEN_smiles_2,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,355.822,2.8788,1.0,5.0,AUTOGEN_smiles_3,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,440.253,4.2983,2.0,5.0,AUTOGEN_smiles_1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,442.496,5.8693,2.0,5.0,AUTOGEN_smiles_5,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,384.270,3.8024,1.0,3.0,AUTOGEN_smiles_4,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,435.907,5.2166,1.0,3.0,AUTOGEN_smiles_96,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
96,401.462,4.6111,2.0,3.0,AUTOGEN_smiles_97,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
97,407.491,4.6726,2.0,4.0,AUTOGEN_smiles_98,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
98,429.259,5.4476,2.0,4.0,AUTOGEN_smiles_99,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
df = full_df.sample(10000, random_state=42)
selection = ['buildingblock1_smiles']
df_selection = df[selection]
df_selection.to_csv('buildingblock1_smiles.smi', sep='\t', index=False, header=False)

In [36]:
# !java -Xms1G -Xmx1G -Djava.awt.headless=true -jar ./PaDEL-Descriptor/PaDEL-Descriptor.jar -removesalt -standardizenitro -fingerprints -descriptortypes ./PaDEL-Descriptor/PubchemFingerprinter.xml -dir ./ -file descriptors_output.csv -maxcpdperfile 0 -threads -1 -usefilenameasmolname buildingblock1_smiles.smi -waitingjobs -1
!java -Djava.awt.headless=true -jar ./PaDEL-Descriptor/PaDEL-Descriptor.jar -removesalt -standardizenitro -fingerprints -descriptortypes ./PaDEL-Descriptor/PubchemFingerprinter.xml -dir ./ -file descriptors_output.csv -maxcpdperfile 0 -threads -1 -usefilenameasmolname buildingblock1_smiles.smi -waitingjobs 200

Processing buildingblock1_smiles_1 in buildingblock1_smiles.smi (1/10000). 
Processing buildingblock1_smiles_2 in buildingblock1_smiles.smi (2/10000). 
Processing buildingblock1_smiles_3 in buildingblock1_smiles.smi (3/10000). 
Processing buildingblock1_smiles_4 in buildingblock1_smiles.smi (4/10000). 
Processing buildingblock1_smiles_6 in buildingblock1_smiles.smi (6/10000). 
Processing buildingblock1_smiles_5 in buildingblock1_smiles.smi (5/10000). 
Processing buildingblock1_smiles_7 in buildingblock1_smiles.smi (7/10000). 
Processing buildingblock1_smiles_8 in buildingblock1_smiles.smi (8/10000). 
Processing buildingblock1_smiles_9 in buildingblock1_smiles.smi (9/10000). 
Processing buildingblock1_smiles_10 in buildingblock1_smiles.smi (10/10000). 
Processing buildingblock1_smiles_11 in buildingblock1_smiles.smi (11/10000). 
Processing buildingblock1_smiles_12 in buildingblock1_smiles.smi (12/10000). 
Processing buildingblock1_smiles_13 in buildingblock1_smiles.smi (13/10000). 
Proc

In [ ]:
Wall time: 54.9 s

In [21]:
pd.read_csv('descriptors_output.csv')

,Name,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880


# padelpy

In [35]:
from padelpy import from_smiles

In [38]:
# calculate molecular descriptors for propane
descriptors = from_smiles('CCC')
print(descriptors)

{'nAcid': '0', 'ALogP': '-0.1853999999999998', 'ALogp2': '0.034373159999999923', 'AMR': '14.709', 'apol': '10.614344000000001', 'naAromAtom': '0', 'nAromBond': '0', 'nAtom': '11', 'nHeavyAtom': '3', 'nH': '8', 'nB': '0', 'nC': '3', 'nN': '0', 'nO': '0', 'nS': '0', 'nP': '0', 'nF': '0.0', 'nCl': '0.0', 'nBr': '0.0', 'nI': '0.0', 'nX': '0.0', 'ATS0m': '440.92087500000014', 'ATS1m': '385.3849459999999', 'ATS2m': '272.44744900000023', 'ATS3m': '84.835296', 'ATS4m': '9.144576', 'ATS5m': '0.0', 'ATS6m': '0.0', 'ATS7m': '0.0', 'ATS8m': '0.0', 'ATS0v': '1519.2206606954308', 'ATS1v': '1764.9267344628088', 'ATS2v': '1788.4692764710278', 'ATS3v': '1061.4246313942745', 'ATS4v': '279.75369987837234', 'ATS5v': '0.0', 'ATS6v': '0.0', 'ATS7v': '0.0', 'ATS8v': '0.0', 'ATS0e': '76.36925999999998', 'ATS1e': '72.02208800000001', 'ATS2e': '125.74608399999998', 'ATS3e': '123.32735999999997', 'ATS4e': '60.46617599999999', 'ATS5e': '0.0', 'ATS6e': '0.0', 'ATS7e': '0.0', 'ATS8e': '0.0', 'ATS0p': '11.9236032387

In [4]:
# calculate molecular descriptors for propane and butane
descriptors = from_smiles(['CCC', 'CCCC'])
print(descriptors)

[{'nAcid': '0', 'ALogP': '-0.1853999999999998', 'ALogp2': '0.034373159999999923', 'AMR': '14.709', 'apol': '10.614344000000001', 'naAromAtom': '0', 'nAromBond': '0', 'nAtom': '11', 'nHeavyAtom': '3', 'nH': '8', 'nB': '0', 'nC': '3', 'nN': '0', 'nO': '0', 'nS': '0', 'nP': '0', 'nF': '0.0', 'nCl': '0.0', 'nBr': '0.0', 'nI': '0.0', 'nX': '0.0', 'ATS0m': '440.92087500000014', 'ATS1m': '385.3849459999999', 'ATS2m': '272.44744900000023', 'ATS3m': '84.835296', 'ATS4m': '9.144576', 'ATS5m': '0.0', 'ATS6m': '0.0', 'ATS7m': '0.0', 'ATS8m': '0.0', 'ATS0v': '1519.2206606954308', 'ATS1v': '1764.9267344628088', 'ATS2v': '1788.4692764710278', 'ATS3v': '1061.4246313942745', 'ATS4v': '279.75369987837234', 'ATS5v': '0.0', 'ATS6v': '0.0', 'ATS7v': '0.0', 'ATS8v': '0.0', 'ATS0e': '76.36925999999998', 'ATS1e': '72.02208800000001', 'ATS2e': '125.74608399999998', 'ATS3e': '123.32735999999997', 'ATS4e': '60.46617599999999', 'ATS5e': '0.0', 'ATS6e': '0.0', 'ATS7e': '0.0', 'ATS8e': '0.0', 'ATS0p': '11.923603238

In [5]:
# in addition to descriptors, calculate PubChem fingerprints
desc_fp = from_smiles('CCC', fingerprints=True)
print(desc_fp)

{'nAcid': '0', 'ALogP': '-0.1853999999999998', 'ALogp2': '0.034373159999999923', 'AMR': '14.709', 'apol': '10.614344000000001', 'naAromAtom': '0', 'nAromBond': '0', 'nAtom': '11', 'nHeavyAtom': '3', 'nH': '8', 'nB': '0', 'nC': '3', 'nN': '0', 'nO': '0', 'nS': '0', 'nP': '0', 'nF': '0.0', 'nCl': '0.0', 'nBr': '0.0', 'nI': '0.0', 'nX': '0.0', 'ATS0m': '440.92087500000014', 'ATS1m': '385.3849459999999', 'ATS2m': '272.44744900000023', 'ATS3m': '84.835296', 'ATS4m': '9.144576', 'ATS5m': '0.0', 'ATS6m': '0.0', 'ATS7m': '0.0', 'ATS8m': '0.0', 'ATS0v': '1519.2206606954308', 'ATS1v': '1764.9267344628088', 'ATS2v': '1788.4692764710278', 'ATS3v': '1061.4246313942745', 'ATS4v': '279.75369987837234', 'ATS5v': '0.0', 'ATS6v': '0.0', 'ATS7v': '0.0', 'ATS8v': '0.0', 'ATS0e': '76.36925999999998', 'ATS1e': '72.02208800000001', 'ATS2e': '125.74608399999998', 'ATS3e': '123.32735999999997', 'ATS4e': '60.46617599999999', 'ATS5e': '0.0', 'ATS6e': '0.0', 'ATS7e': '0.0', 'ATS8e': '0.0', 'ATS0p': '11.9236032387

In [6]:
# only calculate fingerprints
fingerprints = from_smiles('CCC', fingerprints=True, descriptors=False)
print(fingerprints)

{'PubchemFP0': '1', 'PubchemFP1': '1', 'PubchemFP2': '0', 'PubchemFP3': '0', 'PubchemFP4': '0', 'PubchemFP5': '0', 'PubchemFP6': '0', 'PubchemFP7': '0', 'PubchemFP8': '0', 'PubchemFP9': '1', 'PubchemFP10': '0', 'PubchemFP11': '0', 'PubchemFP12': '0', 'PubchemFP13': '0', 'PubchemFP14': '0', 'PubchemFP15': '0', 'PubchemFP16': '0', 'PubchemFP17': '0', 'PubchemFP18': '0', 'PubchemFP19': '0', 'PubchemFP20': '0', 'PubchemFP21': '0', 'PubchemFP22': '0', 'PubchemFP23': '0', 'PubchemFP24': '0', 'PubchemFP25': '0', 'PubchemFP26': '0', 'PubchemFP27': '0', 'PubchemFP28': '0', 'PubchemFP29': '0', 'PubchemFP30': '0', 'PubchemFP31': '0', 'PubchemFP32': '0', 'PubchemFP33': '0', 'PubchemFP34': '0', 'PubchemFP35': '0', 'PubchemFP36': '0', 'PubchemFP37': '0', 'PubchemFP38': '0', 'PubchemFP39': '0', 'PubchemFP40': '0', 'PubchemFP41': '0', 'PubchemFP42': '0', 'PubchemFP43': '0', 'PubchemFP44': '0', 'PubchemFP45': '0', 'PubchemFP46': '0', 'PubchemFP47': '0', 'PubchemFP48': '0', 'PubchemFP49': '0', 'PubchemF

In [7]:
# setting the number of threads, this uses one cpu thread to compute descriptors
descriptors = from_smiles(['CCC', 'CCCC'], threads = -1)
print(descriptors)

[{'nAcid': '0', 'ALogP': '-0.1853999999999998', 'ALogp2': '0.034373159999999923', 'AMR': '14.709', 'apol': '10.614344000000001', 'naAromAtom': '0', 'nAromBond': '0', 'nAtom': '11', 'nHeavyAtom': '3', 'nH': '8', 'nB': '0', 'nC': '3', 'nN': '0', 'nO': '0', 'nS': '0', 'nP': '0', 'nF': '0.0', 'nCl': '0.0', 'nBr': '0.0', 'nI': '0.0', 'nX': '0.0', 'ATS0m': '440.92087500000014', 'ATS1m': '385.3849459999999', 'ATS2m': '272.44744900000023', 'ATS3m': '84.835296', 'ATS4m': '9.144576', 'ATS5m': '0.0', 'ATS6m': '0.0', 'ATS7m': '0.0', 'ATS8m': '0.0', 'ATS0v': '1519.2206606954308', 'ATS1v': '1764.9267344628088', 'ATS2v': '1788.4692764710278', 'ATS3v': '1061.4246313942745', 'ATS4v': '279.75369987837234', 'ATS5v': '0.0', 'ATS6v': '0.0', 'ATS7v': '0.0', 'ATS8v': '0.0', 'ATS0e': '76.36925999999998', 'ATS1e': '72.02208800000001', 'ATS2e': '125.74608399999998', 'ATS3e': '123.32735999999997', 'ATS4e': '60.46617599999999', 'ATS5e': '0.0', 'ATS6e': '0.0', 'ATS7e': '0.0', 'ATS8e': '0.0', 'ATS0p': '11.923603238

In [9]:
# save descriptors to a CSV file
_ = from_smiles('CCC', output_csv='descriptors.csv')
_

{'nAcid': '0',
 'ALogP': '-0.1853999999999998',
 'ALogp2': '0.034373159999999923',
 'AMR': '14.709',
 'apol': '10.614344000000001',
 'naAromAtom': '0',
 'nAromBond': '0',
 'nAtom': '11',
 'nHeavyAtom': '3',
 'nH': '8',
 'nB': '0',
 'nC': '3',
 'nN': '0',
 'nO': '0',
 'nS': '0',
 'nP': '0',
 'nF': '0.0',
 'nCl': '0.0',
 'nBr': '0.0',
 'nI': '0.0',
 'nX': '0.0',
 'ATS0m': '440.92087500000014',
 'ATS1m': '385.3849459999999',
 'ATS2m': '272.44744900000023',
 'ATS3m': '84.835296',
 'ATS4m': '9.144576',
 'ATS5m': '0.0',
 'ATS6m': '0.0',
 'ATS7m': '0.0',
 'ATS8m': '0.0',
 'ATS0v': '1519.2206606954308',
 'ATS1v': '1764.9267344628088',
 'ATS2v': '1788.4692764710278',
 'ATS3v': '1061.4246313942745',
 'ATS4v': '279.75369987837234',
 'ATS5v': '0.0',
 'ATS6v': '0.0',
 'ATS7v': '0.0',
 'ATS8v': '0.0',
 'ATS0e': '76.36925999999998',
 'ATS1e': '72.02208800000001',
 'ATS2e': '125.74608399999998',
 'ATS3e': '123.32735999999997',
 'ATS4e': '60.46617599999999',
 'ATS5e': '0.0',
 'ATS6e': '0.0',
 'ATS7e': 

In [38]:
a = from_smiles(
    smiles='O=C(NCC1CCC(C(=O)O)CC1)OCC1c2ccccc2-c2ccccc21',
    # output_csv='descriptors.csv',
    descriptors=True,
    fingerprints=True,
    timeout=60,
    maxruntime=-1,
    threads=-1
)

In [40]:
print(len(a.keys()))

2756


In [31]:
%%time
from_smiles(
    smiles=df.buildingblock1_smiles.values.tolist(),
    output_csv='descriptors.csv',
    descriptors=True,
    fingerprints=True,
    timeout=60,
    maxruntime=-1,
    threads=-1
)

NameError: name 'from_smiles' is not defined

In [26]:
descriptors_df = pd.read_csv('descriptors.csv')

In [27]:
descriptors_df

,Name,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,AUTOGEN_20241228041453011823341806168_1,1,0.2367,0.056027,53.2410,61.457825,12,13,53,28,...,0,0,0,0,0,0,0,0,0,0
1,AUTOGEN_20241228041453011823341806168_2,1,1.0438,1.089518,65.9434,51.517446,6,6,46,24,...,0,0,0,0,0,0,0,0,0,0
2,AUTOGEN_20241228041453011823341806168_3,1,0.3987,0.158962,33.2160,54.093102,18,19,42,28,...,0,0,0,0,0,0,0,0,0,0
3,AUTOGEN_20241228041453011823341806168_4,1,0.2441,0.059585,32.0907,64.310274,23,25,50,32,...,0,0,0,0,0,0,0,0,0,0
4,AUTOGEN_20241228041453011823341806168_5,1,1.7632,3.108874,65.0602,51.947446,6,6,45,23,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,AUTOGEN_20241228041453011823341806168_140,1,-0.1148,0.013179,49.0313,67.357446,18,19,55,33,...,0,0,0,0,0,0,0,0,0,0
140,AUTOGEN_20241228041453011823341806168_141,1,0.4595,0.211140,42.2190,52.177067,12,13,44,25,...,0,0,0,0,0,0,0,0,0,0
141,AUTOGEN_20241228041453011823341806168_142,1,0.1732,0.029998,46.1197,64.263860,18,19,52,32,...,0,0,0,0,0,0,0,0,0,0
142,AUTOGEN_20241228041453011823341806168_143,1,-0.7669,0.588136,27.5972,58.323481,21,23,47,30,...,0,0,0,0,0,0,0,0,0,0


//////////////////////////////////////////////////////////////////

In [31]:
def lipinski(smiles, df=True):
    # if type(smiles).__name__ == 'DataFrame':
    #     smiles = smiles.values.flatten()

    moldata= []
    # for elem in tqdm(smiles):`
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem)
        moldata.append(mol)

    baseData= np.arange(1,1)
    i=0
    for mol in moldata:

        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)

        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])

        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1

    if df:
        columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]
        descriptors = pd.DataFrame(data=baseData,columns=columnNames)
        return descriptors
    else:
        return baseData

In [32]:
%%timeit
lipinski_df = lipinski(smiles, df=False)

3.68 s ± 148 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
import pandas as pd
import numpy as np
from tqdm import tqdm

def lipinski2(smiles, df=True):
    # Pre-allocate a list for descriptors to avoid using np.vstack in a loop
    descriptors = []
    
    for elem in tqdm(smiles):
        mol = Chem.MolFromSmiles(elem)
        if mol:
            descriptors.append([
                Descriptors.MolWt(mol),
                Descriptors.MolLogP(mol),
                Lipinski.NumHDonors(mol),
                Lipinski.NumHAcceptors(mol)
            ])
        else:
            descriptors.append([None, None, None, None])  # Handle invalid SMILES strings
    
    # Convert directly to a DataFrame or numpy array
    columnNames = ["MW", "LogP", "NumHDonors", "NumHAcceptors"]
    descriptors_array = np.array(descriptors)
    
    if df:
        return pd.DataFrame(data=descriptors_array, columns=columnNames)
    else:
        return descriptors_array

In [24]:
%%timeit
lipinski_df = lipinski2(smiles, df=False)

100%|██████████████████████████████████| 10000/10000 [00:03<00:00, 2909.57it/s]

4.53 s ± 1.27 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
import pandas as pd
import numpy as np
from tqdm import tqdm

def lipinski3(smiles, df=True):
    # Pre-allocate a list for descriptors to avoid using np.vstack in a loop
    descriptors = []
    
    # for elem in tqdm(smiles):
    for elem in smiles:
        mol = Chem.MolFromSmiles(elem)
        descriptors.append([
            Descriptors.MolWt(mol),
            Descriptors.MolLogP(mol),
            Lipinski.NumHDonors(mol),
            Lipinski.NumHAcceptors(mol)
        ])
        
    # Convert directly to a DataFrame or numpy array
    columnNames = ["MW", "LogP", "NumHDonors", "NumHAcceptors"]
    descriptors_array = np.array(descriptors)
    
    if df:
        return pd.DataFrame(data=descriptors_array, columns=columnNames)
    else:
        return descriptors_array

In [30]:
%%timeit
lipinski_df = lipinski3(smiles, df=False)

3.85 s ± 888 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [64]:
from joblib import Parallel, delayed
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import subprocess

def compute_descriptors(smiles):
    # """Compute molecular descriptors for a single SMILES string."""
    mol = Chem.MolFromSmiles(smiles)
    # if mol is None:
    #     return np.array([np.nan, np.nan, np.nan, np.nan])  # Handle invalid SMILES

    desc_MolWt = Descriptors.MolWt(mol)
    desc_MolLogP = Descriptors.MolLogP(mol)
    desc_NumHDonors = Lipinski.NumHDonors(mol)
    desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)

    return np.array([desc_MolWt, desc_MolLogP, desc_NumHDonors, desc_NumHAcceptors])

def lipinski_parallel(smiles, df=True, n_jobs=-1):
    """Compute Lipinski descriptors in parallel."""
    # if isinstance(smiles, pd.DataFrame):
    #     smiles = smiles.values.flatten()

    # Compute descriptors in parallel
    results = Parallel(n_jobs=n_jobs, backend='loky')(
        delayed(compute_descriptors)(s) for s in tqdm(smiles)
    )

    # Stack results into a NumPy array
    baseData = np.vstack(results)

    if df:
        columnNames = ["MW", "LogP", "NumHDonors", "NumHAcceptors"]
        descriptors = pd.DataFrame(data=baseData, columns=columnNames)
        return descriptors
    else:
        return baseData

df = full_df.sample(frac=1, random_state=42)

In [65]:
%%timeit
lipinski_df = lipinski_parallel(df.buildingblock1_smiles, df=False, n_jobs=-1)

  0%|          | 0/1674896 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [76]:
from joblib import Parallel, delayed
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

def compute_descriptors(smiles_chunk):
    results = []
    for smiles in smiles_chunk:
        mol = Chem.MolFromSmiles(smiles)
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
        results.append([desc_MolWt, desc_MolLogP, desc_NumHDonors, desc_NumHAcceptors])
    return results

def lipinski_parallel(smiles, df=True, n_jobs=-1, chunk_size=1000):
    """Compute Lipinski descriptors in parallel."""
    # Split data into chunks
    chunks = [smiles[i:i + chunk_size] for i in range(0, len(smiles), chunk_size)]

    # Compute descriptors in parallel
    results = Parallel(n_jobs=n_jobs, backend='loky')(
        delayed(compute_descriptors)(chunk) for chunk in tqdm(chunks)
    )

    # Flatten the list of results
    flattened_results = [item for sublist in results for item in sublist]

    # Convert to DataFrame or NumPy array
    columnNames = ["MW", "LogP", "NumHDonors", "NumHAcceptors"]
    if df:
        return pd.DataFrame(flattened_results, columns=columnNames)
    else:
        return np.array(flattened_results)

# Example usage
df = full_df.sample(frac=1, random_state=42)
smiles = df.buildingblock1_smiles  # Replace with your SMILES column

In [79]:
descriptors_df = lipinski_parallel(smiles, df=True, n_jobs=-1 , chunk_size=10000)

  0%|          | 0/168 [00:00<?, ?it/s]

In [80]:
descriptors_df

,MW,LogP,NumHDonors,NumHAcceptors
0,379.456,4.4161,2,3
1,355.822,2.8788,1,5
2,440.253,4.2983,2,5
3,442.496,5.8693,2,5
4,384.270,3.8024,1,3
...,...,...,...,...
1674891,387.230,2.6855,1,5
1674892,469.459,5.6299,2,3
1674893,447.269,3.7677,1,4
1674894,441.405,4.6383,2,3


In [51]:
from joblib import Parallel, delayed
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski, rdmolops, AllChem, rdchem, rdEHTTools, rdMolDescriptors
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from joblib import Parallel, delayed
from concurrent.futures import ThreadPoolExecutor
from padelpy import from_smiles
# from jupyter_datatables import init_datatables_mode
# init_datatables_mode()

def compute_descriptors(smiles_chunk):
    results = []
    for smiles in smiles_chunk:
        mol = Chem.MolFromSmiles(smiles)
        
        # General Chemical Descriptors
        desc_MolWt = Descriptors.MolWt(mol)  # Molecular Weight (MW)
        desc_ExactMolWt = rdMolDescriptors.CalcExactMolWt(mol)  # Exact Molecular Weight
        desc_MolLogP = Descriptors.MolLogP(mol)  # LogP (Hydrophobicity)
        desc_TPSA = rdMolDescriptors.CalcTPSA(mol)  # Topological Polar Surface Area (TPSA)
        desc_NumRotatableBonds = rdMolDescriptors.CalcNumRotatableBonds(mol)  # Number of Rotatable Bonds
        desc_NumHDonors = Lipinski.NumHDonors(mol)  # Number of H-Bond Donors
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)  # Number of H-Bond Acceptors
        desc_FormalCharge = rdmolops.GetFormalCharge(mol)  # Formal Charge
        desc_FractionCSP3 = rdMolDescriptors.CalcFractionCSP3(mol)  # Fraction of Csp3 Carbons
        # -------------------------------------------------------
        
        desc_NumHeavyAtoms = mol.GetNumHeavyAtoms()  # Number of Heavy Atoms
        
        # Functional Group Descriptors
        desc_NumAromaticRings = rdMolDescriptors.CalcNumAromaticRings(mol)  # Number of Aromatic Rings
        desc_NumAliphaticRings = rdMolDescriptors.CalcNumAliphaticRings(mol)  # Number of Aliphatic Rings
        desc_NumSaturatedRings = rdMolDescriptors.CalcNumSaturatedRings(mol)  # Number of Saturated Rings
        desc_NumHeteroatoms = rdMolDescriptors.CalcNumHeteroatoms(mol)  # Number of Heteroatoms
        desc_NumRings = rdMolDescriptors.CalcNumRings(mol)  # Total Number of Rings
        # -------------------------------------------------------
        
        # Shape and Geometry Descriptors
        if mol.GetNumConformers() >= 1:
            desc_MolVolume = AllChem.ComputeMolVolume(mol)  # Molecular Volume
            desc_RadiusOfGyration = rdMolDescriptors.CalcRadiusOfGyration(mol)  # Radius of Gyration
            desc_InertialShapeFactor = rdMolDescriptors.CalcInertialShapeFactor(mol)  # Inertial Shape Factor
        else:
            desc_MolVolume = desc_RadiusOfGyration = desc_InertialShapeFactor = 0
        # -------------------------------------------------------
        
        # Binding-Specific Descriptors
        desc_AromaticProportion = (
            rdMolDescriptors.CalcNumAromaticRings(mol) / mol.GetNumHeavyAtoms()
            if mol.GetNumHeavyAtoms() > 0 else 0
        )  # Aromatic Proportion
        desc_HBondPotential = Lipinski.NumHDonors(mol) + Lipinski.NumHAcceptors(mol)  # Hydrogen Bonding Potential
        desc_Lipophilicity = Descriptors.MolLogP(mol)  # Lipophilicity
        desc_ChargeDistribution = rdmolops.GetFormalCharge(mol)  # Charge Distribution
        # -------------------------------------------------------
        
        # Molecular Fingerprints
        # fp_RDK = AllChem.RDKFingerprint(mol)  # RDK Fingerprint
        # desc_FingerprintBits = sum(fp_RDK.GetOnBits())  # Number of Active Bits in RDK Fingerprint
        # fp_Morgan = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024)  # Morgan Fingerprint
        # desc_MorganBits = sum(fp_Morgan)  # Number of Active Bits in Morgan Fingerprint
        # -------------------------------------------------------
        
        # Electrostatic Properties
        desc_ElectroNegativity = Descriptors.MolMR(mol)  # Molecular Refractivity (Electronegativity)
        # -------------------------------------------------------

        
        results.append({
            "smiles": smiles,
            
            "MolWt": desc_MolWt,
            "ExactMolWt": desc_ExactMolWt,
            "MolLogP": desc_MolLogP,
            "TPSA": desc_TPSA,
            "NumRotatableBonds": desc_NumRotatableBonds,
            "NumHDonors": desc_NumHDonors,
            "NumHAcceptors": desc_NumHAcceptors,
            "FormalCharge": desc_FormalCharge,
            "FractionCSP3": desc_FractionCSP3,
            
            "NumHeavyAtoms": desc_NumHeavyAtoms,
            
            "NumAromaticRings": desc_NumAromaticRings,
            "NumAliphaticRings": desc_NumAliphaticRings,
            "NumSaturatedRings": desc_NumSaturatedRings,
            "NumHeteroatoms": desc_NumHeteroatoms,
            "NumRings": desc_NumRings,
            
            "MolVolume": desc_MolVolume,
            "RadiusOfGyration": desc_RadiusOfGyration,
            "InertialShapeFactor": desc_InertialShapeFactor,
            
            "AromaticProportion": desc_AromaticProportion,
            "HBondPotential": desc_HBondPotential,
            "Lipophilicity": desc_Lipophilicity,
            "ChargeDistribution": desc_ChargeDistribution,
            
            # "FingerprintBits": desc_FingerprintBits,
            # "MorganBits": desc_MorganBits,
            
            "ElectroNegativity": desc_ElectroNegativity
        })

        
    return results

def lipinski_parallel(smiles, df=True, n_jobs=-1, chunk_size=1000):
    """Compute Lipinski descriptors in parallel."""
    # Split data into chunks
    chunks = [smiles[i:i + chunk_size] for i in range(0, len(smiles), chunk_size)]

    # Compute descriptors in parallel
    results = Parallel(n_jobs=n_jobs, backend='loky')(
        delayed(compute_descriptors)(chunk) for chunk in tqdm(chunks)
    )

    # Flatten the list of results
    flattened_results = [item for sublist in results for item in sublist]
    
    if df:
        return pd.DataFrame(flattened_results)
    else:
        return np.array(flattened_results)


# Example usage
# df = full_df.sample(frac=1, random_state=42)
smiles = full_df.buildingblock1_smiles

In [19]:
%%time
# %%capture
# df = full_df.sample(frac=1)
# smiles = df.buildingblock1_smiles  # Replace with your SMILES column
# print(smiles.values[:1])
descriptors_df = lipinski_parallel(smiles, df=True, n_jobs=-1 , chunk_size=10000)

  0%|          | 0/168 [00:00<?, ?it/s]

CPU times: user 10.1 s, sys: 3.98 s, total: 14.1 s
Wall time: 48.2 s


In [26]:
pd.set_option('display.max_columns', None)
descriptors_df

,smiles,MolWt,ExactMolWt,MolLogP,TPSA,NumRotatableBonds,NumHDonors,NumHAcceptors,FormalCharge,FractionCSP3,NumHeavyAtoms,NumAromaticRings,NumAliphaticRings,NumSaturatedRings,NumHeteroatoms,NumRings,MolVolume,RadiusOfGyration,InertialShapeFactor,AromaticProportion,HBondPotential,Lipophilicity,ChargeDistribution,ElectroNegativity
0,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,363.413,363.147058,3.7818,75.63,7,2,3,0,0.272727,27,2,1,0,5,3,0,0,0,0.074074,5,3.7818,0,102.3125
1,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,363.413,363.147058,3.7818,75.63,7,2,3,0,0.272727,27,2,1,0,5,3,0,0,0,0.074074,5,3.7818,0,102.3125
2,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,363.413,363.147058,3.7818,75.63,7,2,3,0,0.272727,27,2,1,0,5,3,0,0,0,0.074074,5,3.7818,0,102.3125
3,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,363.413,363.147058,3.7818,75.63,7,2,3,0,0.272727,27,2,1,0,5,3,0,0,0,0.074074,5,3.7818,0,102.3125
4,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,363.413,363.147058,3.7818,75.63,7,2,3,0,0.272727,27,2,1,0,5,3,0,0,0,0.074074,5,3.7818,0,102.3125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1674891,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,380.404,380.148455,4.0688,124.39,8,2,4,0,0.300000,28,2,1,0,8,3,0,0,0,0.071429,6,4.0688,0,102.8755
1674892,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,380.404,380.148455,4.0688,124.39,8,2,4,0,0.300000,28,2,1,0,8,3,0,0,0,0.071429,6,4.0688,0,102.8755
1674893,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,380.404,380.148455,4.0688,124.39,8,2,4,0,0.300000,28,2,1,0,8,3,0,0,0,0.071429,6,4.0688,0,102.8755
1674894,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,380.404,380.148455,4.0688,124.39,8,2,4,0,0.300000,28,2,1,0,8,3,0,0,0,0.071429,6,4.0688,0,102.8755


In [30]:
descriptors_df.smiles[-5:].values

array(['[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O',
       '[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O',
       '[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O',
       '[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O',
       '[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O'],
      dtype=object)

In [7]:
descriptors_df

array([[363.413     , 363.14705815,   3.7818    , ...,   3.7818    ,
          0.        , 102.3125    ],
       [363.413     , 363.14705815,   3.7818    , ...,   3.7818    ,
          0.        , 102.3125    ],
       [363.413     , 363.14705815,   3.7818    , ...,   3.7818    ,
          0.        , 102.3125    ],
       ...,
       [380.404     , 380.14845512,   4.0688    , ...,   4.0688    ,
          0.        , 102.8755    ],
       [380.404     , 380.14845512,   4.0688    , ...,   4.0688    ,
          0.        , 102.8755    ],
       [380.404     , 380.14845512,   4.0688    , ...,   4.0688    ,
          0.        , 102.8755    ]])

In [8]:
descriptors_df[0,:]

array([3.63413000e+02, 3.63147058e+02, 3.78180000e+00, 7.56300000e+01,
       7.00000000e+00, 2.00000000e+00, 3.00000000e+00, 0.00000000e+00,
       2.72727273e-01, 2.70000000e+01, 2.00000000e+00, 1.00000000e+00,
       0.00000000e+00, 5.00000000e+00, 3.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 7.40740741e-02, 5.00000000e+00,
       3.78180000e+00, 0.00000000e+00, 1.02312500e+02])

In [11]:
for i,v in enumerate(descriptors_df[0,:]):
    print(i, "=", v)

0 = 363.41300000000007
1 = 363.147058152
2 = 3.7818000000000023
3 = 75.63
4 = 7.0
5 = 2.0
6 = 3.0
7 = 0.0
8 = 0.2727272727272727
9 = 27.0
10 = 2.0
11 = 1.0
12 = 0.0
13 = 5.0
14 = 3.0
15 = 0.0
16 = 0.0
17 = 0.0
18 = 0.07407407407407407
19 = 5.0
20 = 3.7818000000000023
21 = 0.0
22 = 102.31250000000004


In [67]:
def compute_descriptors2(smiles_chunk):
    
    padels = from_smiles(
        smiles=list(smiles_chunk),
        # output_csv='descriptors.csv',
        descriptors=True,
        fingerprints=True,
        timeout=60,
        maxruntime=-1,
        threads=-1
    )
    
    def convert_value(value):
        try:
            return int(value)
        except ValueError:
            try:
                return float(value)
            except ValueError:
                return str(value)
                
    with ThreadPoolExecutor() as executor:
        converted_values = list(executor.map(convert_value, padels.values()))
        return dict(zip(padels.keys(), converted_values))




def lipinski_parallel(smiles, df=True, n_jobs=-1, chunk_size=1000):
    """Compute Lipinski descriptors in parallel."""
    # Split data into chunks
    chunks = [smiles[i:i + chunk_size] for i in range(0, len(smiles), chunk_size)]

    # Compute descriptors in parallel
    results = Parallel(n_jobs=n_jobs, backend='loky')(
        delayed(compute_descriptors2)(chunk) for chunk in tqdm(chunks)
    )

    # Flatten the list of results
    flattened_results = [item for sublist in results for item in sublist]
    
    if df:
        return pd.DataFrame(flattened_results)
    else:
        return np.array(flattened_results)


# Example usage
# df = full_df.sample(frac=1, random_state=42)
smiles = full_df.buildingblock1_smiles.values

In [68]:
chunk_size = 10000
chunks = [smiles[i:i + chunk_size] for i in range(0, len(smiles), chunk_size)]

In [69]:
%%time
# %%capture
# df = full_df.sample(frac=1)
# smiles = df.buildingblock1_smiles  # Replace with your SMILES column
# print(smiles.values[:1])
descriptors_df = lipinski_parallel(smiles, df=True, n_jobs=-1 , chunk_size=10000)

  0%|          | 0/168 [00:00<?, ?it/s]

RuntimeError: PaDEL-Descriptor encountered an error: PaDEL-Descriptor timed out during subprocess call